In [ ]:
#  EfficientNet-B0 기반 이진분류 모델 만들기 + 전이학습

In [ ]:
# 필요한것 import 

import os
import torch
import copy
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from tqdm import tqdm

In [ ]:
import os
import copy
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from tqdm import tqdm
import matplotlib.pyplot as plt
import requests
from io import BytesIO
from PIL import Image


# 디바이스 설정 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")



#이미지 전처리
transform = transform.Compose([
    transforms.Resize((224,224)), # 사이즈 조정 - efficientnet은 기본 224로 맞춰짐, 다 같이 맞춰야 함
    transforms.ToTensor(),        # tensor 변환 numpy 배열을 pytorch tensor로 변환 (0~1)정규화  
    transforms.Normalize([0.485,0.456,0.406],  # 정규화(mean)
                         [0.229.0.224,0.225]) # 정규화(std)
])


# 예측 함수 정의 (외부에서도 사용 가능)
def predict_image_from_url(image_url, threshold=0.5):
    try:
        response = requests.get(image_url)
        img = Image.open(BytesIO(response.content)).convert("RGB")
        img_tensor = transform(img).unsqueeze(0).to(device)

        model.eval()
        with torch.no_grad():
            output = model(img_tensor)
            prob = torch.sigmoid(output).item()

        if prob > threshold:
            print(f"이미지 예측 결과: 햄버거 (확률: {prob:.4f})")
            return True
        else:
            print(f"이미지 예측 결과: 비햄버거 (확률: {prob:.4f})")
            return False

    except Exception as e:
        print(f"예측 실패: {e}")
        return None

# 그래프 저장 함수
def save_training_graph(train_acc_list, val_acc_list, train_loss_list, save_path="training_result.png"):
    epochs = range(1, len(train_acc_list) + 1)
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_acc_list, label='Train Acc')
    plt.plot(epochs, val_acc_list, label='Val Acc')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Accuracy per Epoch')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_loss_list, label='Train Loss', color='red')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss per Epoch')
    plt.legend()

    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

    
if __name__ == "__main__":
    # 데이터 경로
    data_dir = "C:/Users/baby3/OneDrive/바탕 화면/햄버거 테스트용/model_data"    
    
    # 데이터셋 로드
    train_ds = datasets.ImageFolder(os.path.join(data_dir,"train"),transform=transform)
    val_ds = datasets.ImageFolder(os.path.join(data_dir,"val"), transform=transform)
    test_ds = datasets.ImageFolder(os.path.join(data_dir,"test"),transform=transform)

    #배치사이즈 : 한번에 학습하는 이미지 , 일반적으로 16,32,64 
    # shuffle = True는 훈련 데이터의 순서를 매 epoch마다 랜덤하게 섞는다는 뜻
    train_loder = DataLoader(train_ds,batch_size=32, shuffle=True) 
    train_loder = DataLoader(train_ds,batch_size=32, shuffle=False)
    train_loder = DataLoader(train_ds,batch_size=32, shuffle=False)

    # EfficientNet-B0 모델 불러오기 
    model = models.efficientent_b0(pretrained=True)
    model.classifier = nn.Sequential(
        nn.Dropout(0.3),
        nn.Linear(1280, 1))
        # model.classifier[1] = nn.Linear(in_features=1280, out_features=1)  # 이진 분류 
        # 맨 마지막 분류기 바꿔서 ↑ / sigmoid 는 내부적으로 하는것. 
    model = model.to(device)

    # 손실함수 & 옵티마이저 
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    # EarlyStopping & 최고 모델 저장 준비
    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_acc = 0
    early_stop_count = 0
    early_stop_patience = 10  #10번 정체면 저장

    # epoch 저장 (기록용 리스트)
    train_acc_list = []
    val_caa_list = []
    train_loss_list = []

    # 학습 루프
    epochs = 30
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            preds = torch.sigmoid(outputs) > 0.5
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total
        train_acc_list.append(train_acc)
        train_loss_list.append(running_loss)

        # 검증
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
                outputs = model(inputs)
                preds = torch.sigmoid(outputs) > 0.5
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        val_acc = correct / total
        val_acc_list.append(val_acc)

        print(f"Epoch {epoch+1}  Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

        # 최고 성능 모델 저장
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), "best_burger_model.pth")
            print("모델 저장됨 (최고 성능)")
            early_stop_count = 0
        else:
            early_stop_count += 1
            print(f"EarlyStopping 대기 중... ({early_stop_count}/{early_stop_patience})")

        if early_stop_count >= early_stop_patience:
            print("Early stopping 발생!")
            break

    # 최고 성능 모델로 복원
    model.load_state_dict(best_model_wts)
    print("최고 모델로 복원 완료")

    # 테스트 정확도 확인
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
            outputs = model(inputs)
            preds = torch.sigmoid(outputs) > 0.5
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    print(f"최종 테스트 정확도: {correct / total:.4f}")

    # 그래프 저장
    save_training_graph(train_acc_list, val_acc_list, train_loss_list)

In [ ]:
# b0 모델 예측 코드 

In [ ]:
# test_predict.py

from predict import predict_image_from_url

# 예측하고 싶은 이미지 URL
url = "https://example.com/sample.jpg"

# 예측 실행
predict_image_from_url(url)